In [1]:
# This program extends the training data out to the full map of valid pixels.

In [2]:
import numpy as np
from numpy import savetxt
from numpy import loadtxt
import fitsio
import healpy as hp
import matplotlib.pyplot as plt
import Config
from scipy import interpolate as inter
from astropy.table import Table

In [3]:
res =4096
numBins = 100 # This is used to determine how many subbins to use to avoid crashing memory.
conditions = Config.conditions
file = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/'

In [4]:
validPix = fitsio.read(file + 'Valid_'+str(res)+'_Pixels.fits')['PIXEL']

In [5]:
stds = np.loadtxt(file + str(res) + '_Inter_Blue_>26_Standard_Deviations.csv', delimiter = ',') # Standard devs
redStds = np.loadtxt(file + str(res) + '_Inter_Blue_>26_Red_Standard_Deviations.csv', delimiter = ',') # Standard devs of reduced maps
mean = np.loadtxt(file + str(res) + '_Inter_Blue_>26_Means.csv', delimiter = ',') # Means
inds = np.loadtxt(file + str(res) + '_Inter_Blue_>26_Indices.csv', delimiter = ',').astype(int) # Indices of corrections principal comps.

In [6]:
# Ths reads in the feature vector and converts it to floating points.
strFeatVec = loadtxt(file + str(res) + '_Inter_Blue_>26_Feature_Vectors.csv', delimiter = '\t', dtype = object)
featVec = []
for i in range(len(strFeatVec)):
    
    indVec = []
    
    for j in range(len(strFeatVec[i])):
        
        indVec.append(float(strFeatVec[i][j]))
        
    indVec = np.array(indVec)
    featVec.append(indVec)
    
featVec = np.array(featVec, dtype = object)

In [7]:
x = np.linspace(-3, 3, 100)

In [8]:
yValues = fitsio.read(file + str(res) + '_Inter_Blue_>26_y_Values.fits')

In [9]:
# This makes sure we only read in 1% of the full sky at a time to avoid overflow.
binLims = [0]

for i in range(numBins):
    binLims.append(int((len(validPix) - binLims[-1]) / (numBins - i)) + (binLims[-1]))

In [10]:
# This will be used to record extrapolations.
extrMap = []
for i in range(len(inds)):
    extrMap.append([])

In [ ]:
# This will be the corrections map.
corrMap = []

for i in range(len(binLims) - 1):
    
    condMaps = []
    # This loops over every condition file.
    for cond in conditions:
        condMaps.append(fitsio.read(file + str(res) + '_' + cond + '.fits', rows = np.arange(binLims[i], binLims[i + 1]))['SIGNAL'])
        # This reads in the data.
    condMaps = np.array(condMaps, dtype = object)    
    
    stanMaps = []
    for j in range(len(condMaps)):
        stanMaps.append((condMaps[j] - mean[j]) / stds[j])
        # Create the standardized maps, using the means and stds from last time.
    stanMaps = np.array(stanMaps, dtype = object)
    
    redMaps = np.matmul(featVec, stanMaps)
    # This created the reduced maps using PCA and then standardizes them with standard deviations from training.
    for j in range(len(redMaps)):
        redMaps[j] = redMaps[j] / redStds[j]
    
    sectCorr = np.ones(len(stanMaps[0]))
    # This is the corrections for this specific section of pixels read in.
    
    for j in range(len(inds)):
        
        # Finds all places where extrapolation is necessary and marks them
        extrapolation = np.zeros(len(redMaps[inds[j]]))
        extrapolation[np.where((redMaps[inds[j]] > 3) | (redMaps[inds[j]] < -3))[0]] = 1
        
        # Generates y values
        y = yValues[str(inds[j])]
        
        # Generates the function via extrapolation
        f = inter.interp1d(x, y, bounds_error = False, fill_value = (y[0], y[-1]))
        
        # Generates the inverse correction
        corr = f(redMaps[inds[j]].astype('float'))

        corr = 1 / corr
        
        # Applies the correction to this section.
        sectCorr = sectCorr * corr
        
        # For this index, expands out the extrapolation map.
        extrMap[j].extend(extrapolation)
        
    corrMap.extend(sectCorr)
    
corrMap = np.array(corrMap)
# Everything past here is testing and writing to files.

In [ ]:
import healsparse as hsp
import skyproj

In [ ]:
correctionMap = np.full(12*(res**2), hp.UNSEEN)
correctionMap[validPix] = corrMap
corrHSP = hsp.HealSparseMap(nside_coverage=32, healpix_map=correctionMap)

In [ ]:
# The default DES projection is a McBrydeSkymap.
fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111)
sp = skyproj.DESSkyproj(ax=ax)
sp.draw_des(label='DES')
sp.draw_hspmap(corrHSP, cmap = 'viridis')
plt.clim(0.9, 1.2)
plt.colorbar(location = 'bottom', label = 'Multiplicative Correction')
fig.suptitle('Correction Map')
sp.legend()
plt.show()

In [ ]:
extrapolationMap = np.full(12*(res**2), hp.UNSEEN)
extrapolationMap[validPix] = extrMap[0]
extrHSP = hsp.HealSparseMap(nside_coverage=32, healpix_map=extrapolationMap)

In [ ]:
# The default DES projection is a McBrydeSkymap.
fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111)
sp = skyproj.DESSkyproj(ax=ax, linestyle = '--', linewidth = 0.5)
sp.draw_des(label='DES')
sp.draw_hspmap(extrHSP, cmap = 'Greys')
plt.clim(0, 1)
fig.suptitle('Extrapolation Map')
sp.legend()
plt.show()

In [ ]:
my_table = Table()
my_table['SIGNAL'] = corrMap
my_table.write(file + str(res) + '_Inter_Blue_>26_Corrections_Map.fits', overwrite = True)

In [ ]:
ext_table = Table()
for i in range(len(inds)):
    ext_table[str(inds[i])] = extrMap[i]
ext_table.write(file + str(res) + '_Extrapolation_Blue_>26_Maps.fits', overwrite = True)

In [ ]:
corrMap = fitsio.read(file + str(res) + '_Inter_Blue_>26_Corrections_Map.fits')['SIGNAL']

In [ ]:
correctionMap = np.full(12*(res**2), hp.UNSEEN)
correctionMap[validPix] = corrMap
corrHSP = hsp.HealSparseMap(nside_coverage=32, healpix_map=correctionMap)

In [ ]:
# The default DES projection is a McBrydeSkymap.
fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111)
sp = skyproj.DESSkyproj(ax=ax)
sp.draw_hspmap(corrHSP, cmap = 'viridis')
plt.clim(0.9, 1.2)
plt.colorbar(location = 'bottom', label = 'Multiplicative Correction')
fig.suptitle('Correction Map')
plt.show()